# SETUP

In [1]:
from acqpack import Manifold, Mfcs
from acqpack import Motor, AsiController, Autosampler, FractionCollector
from acqpack import utils as ut
from acqpack import gui
import pandas as pd
import time

In [2]:
p = Mfcs('config/mfcs/mfcs.yaml', 'config/mfcs/chanmap.txt')

MFCS initialized. SN: 1149
PID alpha: 2
Pressure units: psi


In [3]:
m = Manifold("192.168.1.3", 'config/manifold/valvemap.txt')
# gui.manifold_control(m)

In [4]:
a = Autosampler(Motor('config/sampler/motor.yaml'),  AsiController('config/sampler/asi.yaml'))
a.add_frame('wash', 'config/sampler/deck.txt', 'config/sampler/4scint.txt')
a.add_frame('plate', 'config/sampler/deck.txt', 'config/sampler/96plate.txt')
gui.stage_control(a)

(-41.447, -2.163, 0.000) hardware
(2.493, 12.749, 99.600) wash
(2.493, 12.749, 99.600) plate


In [29]:
a.goto('wash', 'contents', 'waste', zh_travel=30)

In [ ]:
a.goto('plate','well','A01')

In [ ]:
a.goto('wash','xyz',(0,0,50))

In [5]:
f = FractionCollector(AsiController('config/collector/asi.yaml'))
f.add_frame('plate', 'config/collector/deck.txt', 'config/collector/48vial.txt')
gui.stage_control(f)

(0.000, 0.000) hardware
(0.000, 0.000) plate


In [24]:
f.add_frame('plate', 'config/collector/deck.txt', 'config/collector/48vial.txt')


In [7]:
f.goto('plate','contents','waste1')

In [ ]:
# !!!! Also must have MM folder on system PATH

# mm_version = 'C:\Program Files\Micro-Manager-1.4'
# cfg = r'C:\Users\4DICE\Documents\beadsynthesis_withstage_nofraction.cfg'
# import sys
# sys.path.insert(0, mm_version) # make it so python can find MMCorePy
# import MMCorePy
# from PIL import Image
# core = MMCorePy.CMMCore()
# core.loadSystemConfiguration(cfg)

In [ ]:
# gui.video(core)

In [8]:
# FUNCTIONS
def timestamp():
    return time.strftime("%Y%m%d_%H:%M:%S", time.localtime())

def log(msg):
    msg = ' '.join([timestamp(), msg])
    l.append(msg)
    print msg
    
    
def plc(i):
    if i==1:
        m.close('name', 'plc')
    else:
        m.open('name', 'plc')
        
def bulb(i):
    if i==1:
        m.close('name', 'lamp')
    else:
        m.open('name', 'lamp')
    
def shutter(i):
    if i==1:
        m.close('name', 'shutter')
    else:
        m.open('name', 'shutter')        
            

def init_valves():
    m.close('name', 'drops')
    m.close('name', 'mix_waste')
    m.close('name', 'waste')

    m.close('input', 'wash')
    m.close('input', 'sampler')
    
    m.close('name', 'r0')
    m.close('name', 'r1') 
    m.open('name', 'r2') # use r2

def close_valves():
    m.close('name', 'drops')
    m.close('name', 'mix_waste')
    m.close('name', 'waste')

    m.close('input', 'wash')
    m.close('input', 'sampler')
    
    m.close('name', 'r0')
    m.close('name', 'r1')
    m.close('name', 'r2')
    
    
def sheath():
    p.pid(4,5)
    p.set('name','sheath', 25)
    time.sleep(2.5)
    p.set('name', 'sheath', 0)
    time.sleep(3.5)

In [11]:
sheath()

# SCRIPT

- 255 um diameter tubing, ~ 35-40 cm
- see platemap
- Oil: 4.19 psi
- In2: 0.87 psi

In [26]:
def setup(sec_bulb_min, sec_purge, code_blank):
    log('SETUP')
    shutter(0)
    plc(1)
    bulb(1)
    t_start = time.time()
    
    a.home()
    log('sampler at home')
    f.goto('plate', 'contents', 'waste1')
    log('collector at plate::name=waste1')

    init_valves()

    flush(30,30)
    
    log('purging air {} sec'.format(sec_purge))
    m.open('input','wash')
    m.open('name', 'waste')
    time.sleep(sec_purge)
    init_valves()
    
    t_elapsed = time.time() - t_start
    t_remaining = sec_bulb_min - t_elapsed
    
    
    if t_remaining < 0:
        log('setup in {} sec; bulb is ready ({} minimum)'.format(t_elapsed, sec_bulb_min))
        t_remaining = 0
    else:
        log('setup in {} sec; bulb ready in {} additional sec'.format(t_elapsed, t_remaining)) 
    time.sleep(t_remaining)
    
    # manually adjust droplet pressures
    prime(code_blank, 30)
    log('loading blank code...')
    
    for k, v in p_table.loc['beads'].iteritems():
        p.set('name', k, v) 
    m.open('input','sampler')
    log('manually adjusting pressures...')
    raw_input('position scope to OUT; press enter when drops are stable')
    p_table.loc[['prime','beads'],'oil'] = p.read('name', 'oil')
    p_table.loc['beads','sampler'] = p.read('name', 'sampler')

    flush(30,30)

    

# sample_n -> waste   
def prime(code, sec):
    log('{} PRIME {} {}'.format(code.name, code['well'], code['code']))
    a.goto('plate', 'n', code['n'], zh_travel=30)
    log('sampler at plate::well={} c={}'.format(code['well'], code['code']))
    for k, v in p_table.loc['prime'].iteritems():
        p.set('name', k, v)
    
    m.open('name', 'mix_waste')
    m.open('name', 'waste')
    m.open('input', 'sampler')
    time.sleep(sec)
    
    init_valves()

    
# sample_n -> outlet
def beads(code, vial, sec_stabilize, sec_polymerize):
    # switch collector wells
    sheath()
    
    log('{} BEADS {} {}'.format(code.name, code['well'], code['code']))
    for k, v in p_table.loc['beads'].iteritems():
        p.set('name', k, v)
    
    f.goto('plate', 'vial', vial)
    log('collector at plate::vial={} c={}'.format(vial, code['code']))
    collect = f.frames['plate'].position_table
    collect.loc[collect.vial==vial, 'contents'] = code['code']  # record contents
    
    m.open('input','sampler')
    m.open('name', 'drops')
    log('stabilizing vial={} {} for {} sec'.format(vial, code['code'], sec_stabilize))
    time.sleep(sec_stabilize)
    
    log('polymerizing vial={} {} for {} sec'.format(vial, code['code'], sec_polymerize))
    shutter(1)
    time.sleep(sec_polymerize)
    shutter(0)
    
    init_valves()


# wash -> sampler
# meanwhile: flush outlet       
# wash -> waste 
# meanwhile: wash needle
# meanwhile: flush outlet
def flush(sec_bkflush=30, sec_fwdflush=30):
    log('FLUSH')
    for k, v in p_table.loc['flush'].iteritems():
        p.set('name', k, v)

    # briefly fwd flush
    m.open('name', 'mix_waste')
    m.open('name', 'waste')
    m.open('input', 'wash')
    time.sleep(2)
    init_valves()

    # increase oil flow rate (do after init_valves() above)
    m.close('name', 'r2')
    m.open('name', 'r0')

    # bck flush
    log('flushing back {} sec'.format(sec_bkflush))
    a.goto('wash', 'contents', 'waste')
    log('sampler at wash::contents=water')
    m.open('input', 'wash')
    m.open('input', 'sampler')
    time.sleep(sec_bkflush)
    m.close('input', 'sampler')

    # fwd flush
    log('flushing forward {} sec'.format(sec_fwdflush))
    m.open('name', 'mix_waste')
    m.open('name', 'waste')
    # meanwhile: wash needle
    t_start = time.time()
    a.goto('wash', 'contents', 'ipa', zh_travel=30)
    log('sampler at wash::contents=ipa')
    time.sleep(4) # pause briefly in ipa
    a.goto('wash', 'contents', 'water', zh_travel=30)
    log('sampler at wash::contents=water')
    t_elapsed = time.time() - t_start
    t_remaining = sec_fwdflush - t_elapsed
    if t_remaining < 0:
        log('desired flush_fwd of {} sec insufficient for needle wash; flushed {} sec'.format(sec_fwdflush, t_elapsed))
        t_remaining = 0
    time.sleep(t_remaining)
    
    # toggle drops
    m.close('name', 'r0')
    m.open('name', 'drops')
    time.sleep(1)

    init_valves()
    sheath()

    

def cleanup(bulb_state=0):
    log('CLEANUP')
    bulb(bulb_state)
    f.goto('plate', 'contents', 'waste2')
    log('collector at plate::name={}'.format('waste2'))

    flush(20,20)
    
    a.goto('wash', 'contents', 'waste')
    log('sampler at wash::name={}'.format('waste'))
    
    close_valves()
    for k, _ in p_table.loc['beads'].iteritems():
        p.set('name', k, 0.15) 

    # save
    p_table.to_csv('pressures.csv', sep='\t')  # pressures
    f.frames['plate'].position_table.to_csv('collected_plate.csv', index=False, sep='\t')  # collection record
    pd.DataFrame(l).to_csv('log.txt', index=False, header=False) # log

    log('DONE')

In [13]:
platemap = a.frames['plate'].position_table
codemap = platemap.loc[platemap['series'].notnull()].reset_index(drop=True)  # rows of platemap that have a code
codemap

,n,s,r,c,well,code,series,x,y,z
0,0,0,0,0,A01,eu,0.0,2.493,12.749,-0.85
1,1,1,0,1,A02,dy,0.0,11.493,12.749,-0.85
2,2,2,0,2,A03,sm,0.0,20.493,12.749,-0.85
3,41,42,3,5,D06,eu,1.0,47.493,39.749,-0.85
4,42,41,3,6,D07,dy,1.0,56.493,39.749,-0.85
5,43,40,3,7,D08,sm,1.0,65.493,39.749,-0.85
6,93,86,7,9,H10,eu,2.0,83.493,75.749,-0.85
7,94,85,7,10,H11,dy,2.0,92.493,75.749,-0.85
8,95,84,7,11,H12,sm,2.0,101.493,75.749,-0.85


In [30]:
code_blank = platemap[platemap.code=='bk'].iloc[1]
code_blank

n             13
s             22
r              1
c              1
well         B02
code          bk
series       NaN
x         11.493
y         21.749
z          -0.85
Name: 13, dtype: object

In [27]:
closed_pressure = 1.0
p_table = pd.DataFrame(columns=['wash','sampler','oil'], index=['prime','beads','flush']).fillna(closed_pressure)
p_table.loc['prime','sampler'] = 13.0
p_table.loc[['prime','beads'],'oil'] = 1.53  # beads pressure (will be adjusted on i=0)
p_table.loc['beads','sampler'] = .5  # beads pressure (will be adjusted on i=0)
p_table.loc['flush','wash'] = 13.0
p_table.loc['flush','oil'] = 15.0
p_table

,wash,sampler,oil
prime,1.0,13.0,1.53
beads,1.0,0.5,1.53
flush,13.0,1.0,15.00


In [31]:
# RUN!!!
l = []  # log
vial_start = 1 # starting collector vial

setup(0,5, code_blank)

for i, code in codemap.iterrows():
    vial = vial_start + i  # code.name = i; vial where the fc will collect
    prime(code, 30)
    beads(code, vial, 40, 60*5)
    flush(25, 30)

cleanup(0)

20171107_14:09:03 SETUP
20171107_14:09:08 sampler at home
20171107_14:09:12 collector at plate::name=waste1
20171107_14:09:12 FLUSH
20171107_14:09:14 flushing back 30 sec
20171107_14:09:18 sampler at wash::contents=water
20171107_14:09:48 flushing forward 30 sec
20171107_14:09:56 sampler at wash::contents=ipa
20171107_14:10:08 sampler at wash::contents=water
20171107_14:10:26 purging air 5 sec
20171107_14:10:31 setup in 87.5009999275 sec; bulb is ready (0 minimum)
20171107_14:10:31 13 PRIME B02 bk
20171107_14:10:44 sampler at plate::well=B02 c=bk
20171107_14:11:14 loading blank code...
20171107_14:11:14 manually adjusting pressures...
position scope to OUT; press enter when drops are stable
20171107_14:15:52 FLUSH
20171107_14:15:54 flushing back 30 sec
20171107_14:16:07 sampler at wash::contents=water
20171107_14:16:37 flushing forward 30 sec
20171107_14:16:45 sampler at wash::contents=ipa
20171107_14:16:57 sampler at wash::contents=water
20171107_14:17:15 0 PRIME A01 eu
20171107_14:17

In [26]:
l=[]
flush(30,30)

20171106_17:45:38 FLUSH
20171106_17:45:40 flushing back 30 sec
20171106_17:45:45 sampler at wash::contents=water
20171106_17:46:15 flushing forward 30 sec
20171106_17:46:23 sampler at wash::contents=ipa
20171106_17:46:35 sampler at wash::contents=water


In [23]:
a.frames.wash.position_table

,n,s,r,c,pos,contents,x,y,z
0,0,0,0,0,A01,waste,8.192,-18.952,37.7
1,1,1,0,1,A02,ipa,36.567,-18.952,37.7
2,2,2,0,2,A03,water,64.942,-18.952,37.7
3,3,3,0,3,A04,NaN,93.317,-18.952,37.7


In [24]:
a.goto('wash', 'pos', 'A01', zh_travel=30)

In [17]:
for k, v in p_table.loc['beads'].iteritems():
    p.set('name', k, v)
a.goto('plate', 'n', 13, zh_travel=30)


In [67]:
bulb(0)

In [24]:
del p

Closed connection to device with SN 1149
MFCS library released


input: 25 ul
output: 20 ul

5 psi wash (tygon with pin)
- backflush: 3.4 ul/sec * 20 sec = 68 ul
- forward flush: 3.4 ul/sec * 20 sec = 68 ul

7.5 psi sampler (255 um dia)
- prime: ~4.75 ul/sec * 12 sec = 57 ul

15 psi r0 (tygon with pin)
- flush: 1.9 ul/sec * 40 = 76 ul